### 查询优化
    1.完善问题
        不断引导用户完善访问信息
    2.多路召回 MultiQueryRetriever
        将原问题生成多个问题，然后用这些问题去向量检索
        相当于：一次查询 → 多次检索 → 合并结果 → 去重
    3.问题分解
        将问题拆解为子问题
        根据子问题进行文档检索
        根据检索到的文档回答原问题
    4.上位优化
        生成更抽象的上位问题
        结合上位问题和原问题获取更准确答案
    5.假设性文档嵌入
        根据文档生成假设性答案
        根据假设答案向量去向量检索
        根据检索到的文档回答原问题
    6.混合检索
        基于向量检索文档
        基于关键词检索文档
        融合检索结果  EnsembleRetriever(retrievers=[BM25_retriever, vector_retriever], weights=[0.5, 0.5])
        根据混合检索结果回答问题

In [ ]:
# 1.完善问题
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1.模型初始化
llm = ChatTongyi(model="qwen-max")
embeddings_model = DashScopeEmbeddings(model="text-embedding-v1")


# 格式化输出内容
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i + 1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# 2.意图识别
# 2.1 示例业务模板
templates = {
    "订机票": ["起点", "终点", "时间", "座位等级", "座位偏好"],
    "订酒店": ["城市", "入住日期", "退房日期", "房型", "人数"],
}

# 2.2 意图识别提示模板
intent_prompt = PromptTemplate(
    input_variables=["user_input", "templates"],
    template="根据用户输入 '{user_input}'，选择最合适的业务模板。可用模板如下：{templates}。请返回模板名称。"
)

# 2.3 创建意图识别链
intent_chain = intent_prompt | llm

# 2.4 模拟用户输入
user_input = "我想订一张长沙去北京的机票"

# 2.5 识别意图
intent = intent_chain.invoke({"user_input": user_input, "templates": str(list(templates.keys()))}).content
print("意图：", intent)

# 3.优化访问模版
# 3.1 优化模版构建
selected_template = templates.get(intent)
print("模板：", selected_template)

# 补充信息提示模板
info_prompt = f"""
    请根据用户原始问题和模板，判断原始问题是否完善。
    如果问题缺乏需要的信息，请生成一个友好的请求，明确指出需要补充的信息。
    若问题完善后，返回包含所有信息的完整问题。

    ### 原始问题
    {user_input}

    ### 模板
    {",".join(selected_template)}

    ### 输出示例
    {{
        "isComplete": true,
        "content": "`完整问题`"
    }}
    {{
        "isComplete": false,
        "content": "`友好的引导到需要补充信息`"
    }}
"""

print(f"info_prompt: \n {info_prompt}")

# 3.2 基于优化模版访问
# 历史记录
chat_history = ChatMessageHistory()
# 聊天模版
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个信息补充助手，任务是分析用户问题是否完整。"),
        ("placeholder", "{history}"),  # 历史记录的占位
        ("human", "{input}"),
    ]
)

# 补充信息链
info_chain = prompt | llm

# 自动处理历史记录，将记录注入输入并在每次调用后更新它
# 1. 系统会根据当前会话 ID 获取聊天历史（通过提供的 lambda 函数）
# 2. 将当前输入消息（从 input 键获取）和聊天历史（放入 history 键）组合成最终输入
# 3. 调用 info_chain 进行处理
# 4. 通常还会将新的交互记录自动保存到聊天历史中
with_message_history = RunnableWithMessageHistory(
    info_chain,  # 被包装的链式处理对象（通常是一个 LangChain 链或类似的可运行对象）
    lambda session_id: chat_history,  # 消息历史存储的获取函数： 以 session_id 为参数，返回对应的 chat_history 对象
    input_messages_key="input",  # 指定输入消息在输入字典中的键名
    history_messages_key="history",  # 指定历史消息在输入字典中的键名
)

# 判断问题是否完整，如果不完整，则生成追问请求 调用被包装的链（info_chain），并自动处理消息历史的注入和管理。
# 1. input 键：对应 RunnableWithMessageHistory 初始化时设置的 input_messages_key="input"，表示当前用户消息的字段名
# 2. info_prompt：用户的实际输入内容（通常是一个问题或指令字符串）
# 3. session_id：标识对话会话的唯一键，用于从历史存储中获取或更新对话历史，在多用户场景中，通过不同 session_id 隔离各自的对话历史
info_request = with_message_history.invoke(input={"input": info_prompt},
                                           config={"configurable": {"session_id": "unused"}}).content

# 3.3 解析访问结果
parser = JsonOutputParser()
json_data = parser.parse(info_request)
# 循环判断是否完整，并提交用户补充信息
while json_data['isComplete'] is False:
    # 根据大模型的引导，用户补充信息
    user_answer = input(json_data['content'])
    # 提交用户补充信息，并判断问题是否完整
    info_request = with_message_history.invoke(input={"input": user_answer},
                                               config={"configurable": {"session_id": "unused"}}).content

    # 打印完整历史记录 确认是否有存储
    print("=" * 100)
    print("当前对话历史：")
    for message in chat_history.messages:
        print(f"{message.type}: {message.content}")
    print("=" * 100)

    try:
        json_data = parser.parse(info_request)
    except Exception as e:
        print("json parse error")
        break
print(info_request)


In [ ]:
# 2.多路召回 MultiQueryRetriever
# 相当于一次查询 → 多次检索 → 合并结果 → 去重
# 它让 LLM 把原始 query 改写为一组语义不同但相关的问题：
#   比如 LLM 生成：
#       deepseek 遭遇了哪些质疑？
#       哪些国家限制或封禁了 deepseek？
#       deepseek 在国际上面临哪些挑战？
#       deepseek 是否遇到安全类指控？
#   然后 用这4个问题全部去向量检索
import logging
import os

from langchain_chroma import Chroma
from langchain_classic.retrievers import MultiQueryRetriever
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 日志句柄初始化
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)


# 格式化输出内容
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i + 1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


# 1.文件路径
RESOURCE_DIR = "/mnt/c/大模型/智泊大模型全栈教程总结/02-教材整理 L2/代码/Langchain/6.langchain高级RAG/data/resources"
TXT_DOCUMENT_PATH = os.path.join(RESOURCE_DIR, "deepseek百度百科.txt")

# 2.模型初始化
llm = ChatTongyi(model="qwen-max")
embeddings_model = DashScopeEmbeddings(model="text-embedding-v1")

# 3.加载文档
loader = TextLoader(TXT_DOCUMENT_PATH, encoding='utf-8')
docs = loader.load()

# 4.召回引擎创建与数据索引
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=chunks,
                                    embedding=embeddings_model,
                                    collection_name="multi-query")
retriever = vectorstore.as_retriever()

# 5. 多路召回
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""
        You are an AI language model assistant.
        Your task is to generate 5 different versions of the given user
        question to retrieve relevant documents from a vector  database.
        By generating multiple perspectives on the user question,
        your goal is to help the user overcome some of the limitations
        of distance-based similarity search.
        Provide these alternative questions separated by newlines.
        Original question: {question}
    """
)

retrieval_from_llm = MultiQueryRetriever.from_llm(
    prompt=QUERY_PROMPT,
    retriever=retriever,
    llm=llm,
    include_original=True  # 是否包含原始查询
)

# 自动去重
unique_docs = retrieval_from_llm.invoke("详细介绍DeepSeek")
pretty_print_docs(unique_docs)

# 6. 答案合成
# 创建prompt模板
template = """
请根据以下文档回答问题:
### 文档:
{context}
### 问题:
{question}
"""

# 由模板生成prompt
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm
response = chain.invoke({"context": [doc.page_content for doc in unique_docs], "question": "详细介绍DeepSeek"})
print('-'*20 + "答案合成" + '-'*20)
print(response.content)


In [ ]:
# 3.问题分解
from typing import List

from langchain_chroma import Chroma
from langchain_classic.retrievers.multi_query import LineListOutputParser
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate, BasePromptTemplate
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import Runnable, RunnableLambda


# 格式化输出内容
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i + 1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


# 1. 模型初始化
llm = ChatTongyi(model="qwen-max")
embeddings_model = DashScopeEmbeddings(model="text-embedding-v1")

# 2. 数据准备
documents = [
    Document(
        page_content="""番茄炒蛋的食材：\n
        - 新鲜鸡蛋：3-4个（根据人数调整）
        - 番茄：2-3个中等大小\n- 盐：适量
        - 白糖：一小勺（可选，用于提鲜）
        - 食用油：适量
        - 葱花：少许（可选，用于增香）\n
        这些是最基本的材料，当然也可以根据个人口味添加其他调料或配料。
        """),
    Document(
        page_content="""番茄炒蛋的步骤：鸡蛋打入碗中，加入少许盐，用筷子或打蛋器充分搅拌均匀；
           - 番茄洗净后切成小块备用。\n
           3. **炒鸡蛋**：锅内倒入适量食用油加热至温热状态，然后将搅拌好的鸡蛋液缓缓倒入锅中。
           待鸡蛋凝固时轻轻翻动几下，让其受热均匀直至完全熟透，随后盛出备用。\n
           4. **炒番茄**：在同一锅里留下的底油中放入切好的番茄块，中小火慢慢翻炒至出汁，可根据个人口味加一点点白糖提鲜。\n
           5. **合炒**：当番茄炒至软烂并开始释放大量汤汁时，再把之前炒好的鸡蛋倒回锅里，快速与番茄混合均匀，同时加入适量的盐调味。
           如果喜欢的话还可以撒上一些葱花增加香气。\n
           6. **完成**：最后检查一下味道是否合适，确认无误后即可关火装盘享用美味的番茄炒蛋啦！
           """),
    Document(
        page_content="""技巧与注意事项：
        1. **选材**：选择新鲜的鸡蛋和成熟的番茄。新鲜的食材是做好这道菜的基础。
        2. **打蛋液**：将鸡蛋打入碗中后加入少许盐（根据个人口味调整），然后充分搅拌均匀。这样做可以让蛋更加松软且味道更佳。
        3. **处理番茄**：番茄最好先用开水稍微焯一下皮，然后去皮切块。这样可以去除表皮的硬质部分，让番茄更容易入味，并且口感更好。
        4. **热锅冷油**：先用中小火把锅烧热，再倒入适量食用油，待油温五成热时下蛋液。这样的做法可以使蛋快速凝固形成漂亮的形状而不易粘锅。
        5. **分步烹饪**：通常建议先炒鸡蛋至半熟状态取出备用；接着利用剩下的底油继续翻炒番茄至出汁，
        最后再将之前炒好的鸡蛋倒回锅里与番茄混合均匀加热即可。
        6. **调味品**：除了基本的盐之外，还可以根据喜好添加少量糖来提鲜或者一点酱油增色添香。注意调味料不宜过多以免掩盖了食材本身的味道。
        7. **出锅前加葱花**：如果喜欢的话，在即将完成时撒上一些葱花不仅能增加菜品色泽还能增添香气。
        """)
]

# 3. 数据向量化存储
vectorstore = Chroma.from_documents(documents=documents,
                                    embedding=embeddings_model,
                                    collection_name="decomposition")

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
pretty_print_docs(retriever.invoke("番茄炒蛋怎么制作？"))

#  4. 问题分解
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to break down the input question into 3 sub-questions,
     and solve the complete problem by solving these sub-questions one by one.
     The sub-questions need to retrieve relevant documents in the vector database.
     By decomposing the user's question to generate sub-questions,
     your goal is to help users overcome some limitations of distance-based similarity search.
     Provide these sub-questions separated by newlines, no additional content is required. Original question: {question}""",
)

chain = QUERY_PROMPT | llm | LineListOutputParser()
questions = chain.invoke({"question": "番茄炒蛋怎么制作？"})
print('-' * 20 + '分解问题' + '-' * 20)
print(questions)

# 5. 问题分解整合功能类构建
SUB_QUESTION_PROMPT = PromptTemplate(
    input_variables=["question", "sub_question", "documents"],
    template="""
    To address the main problem {question},
    you need to first resolve the sub-question {sub_question}.
    Below is the reference document provided to support your reasoning:\n\n{documents}\n\n
    Please provide the answer to the current sub-question directly.""",
)

class DecompositionQueryRetriever(BaseRetriever):
    # 向量数据库检索器
    retriever: BaseRetriever
    # 生成子问题链
    llm_chain: Runnable
    # 解决子问题链
    sub_llm_chain: Runnable

    @classmethod
    def from_llm(
            cls,
            retriever: BaseRetriever,
            llm: BaseLanguageModel,
            prompt: BasePromptTemplate = QUERY_PROMPT,
            sub_prompt: BasePromptTemplate = SUB_QUESTION_PROMPT
    ) -> "DecompositionQueryRetriever":
        output_parser = LineListOutputParser()
        llm_chain = prompt | llm | output_parser
        sub_llm_chain = sub_prompt | llm
        return cls(
            retriever=retriever,
            llm_chain=llm_chain,
            sub_llm_chain=sub_llm_chain
        )

    def _get_relevant_documents(
            self,
            query: str,
            *,
            run_manager: CallbackManagerForRetrieverRun,
    ) -> List[Document]:
        # 生成子问题
        sub_queries = self.generate_queries(query)
        # 解决子问题
        documents = self.retrieve_documents(query, sub_queries)
        return documents

    def generate_queries(self, question: str) -> List[str]:
        response = self.llm_chain.invoke({"question": question})
        lines = response
        print(f"Generated queries: {lines}")
        return lines

    def retrieve_documents(self, query: str, sub_queries: List[str]) -> List[Document]:
        sub_llm_chain = RunnableLambda(
            # 传入子问题，检索文档并回答
            lambda sub_query: self.sub_llm_chain.invoke(
                {
                    "question": query,
                    "sub_question": sub_query,
                    "documents": [doc.page_content for doc in self.retriever.invoke(sub_query)]
                }
            )
        )
        # 批量执行所有的子问题
        responses = sub_llm_chain.batch(sub_queries)
        # 将子问题和答案合并作为解决主问题的文档
        documents = [
            Document(page_content=sub_query + "\n" + response.content)
            for sub_query, response in zip(sub_queries, responses)
        ]
        return documents


# 6. LangChain问题分解整合测试
decompositionQueryRetriever = DecompositionQueryRetriever.from_llm(llm=llm, retriever=retriever)
decomposition_docs = decompositionQueryRetriever.invoke("番茄炒蛋怎么制作？")
pretty_print_docs(decomposition_docs)

# 8. 根据召回的文档解答问题
# 创建prompt模板
template = """
    请根据以下文档回答问题:
    ### 文档:
    {context}
    ### 问题:
    {question}
"""
# 由模板生成prompt
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

response = chain.invoke({"context": [doc.page_content for doc in decomposition_docs], "question": "番茄炒蛋怎么制作？"})
print('-' * 20 + '大模型回答' + '-' * 20)
print(response.content)

In [ ]:
# 4.上位问题
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi

"""
核心思想
    Step Back 是一种通过让模型先回答更抽象的“上位问题”（step-back question），再基于抽象答案解决原问题的技术。
    其灵感来源于人类思考复杂问题时，会先退一步思考更通用的原则。

关键步骤
1. 生成上位问题：
    从原始问题中提取一个更抽象、更本质的问题。
    示例：
    原问题 → "AlphaGo 如何击败李世石？"
    上位问题 → "强化学习在棋类游戏中的基本原理是什么？"

2. 回答上位问题：
    先获取抽象问题的答案（通用知识）。

3. 结合解决原问题：
    用抽象答案作为上下文，推导出原问题的具体答案。
"""
# 1.模型初始化
llm = ChatTongyi(model="qwen-max")

# 2.上位问题模版
step_back_prompt = PromptTemplate.from_template(
    """
    基于以下问题，生成一个更抽象的上位问题：
    原始问题: {original_question}
    上位问题:
    """
)

# 3. 生成上位问题
question = "AlphaGo 如何击败李世石？"
abstract_answer = llm.invoke(step_back_prompt.format(original_question=question))
print(f"上位问题：\n{abstract_answer}")

# 4.结合两者回答原问题
final_prompt = f"""请基于以下信息回答问题：
    上位问题: {abstract_answer}
    原始问题: {question}
    最终答案:"""
result = llm.invoke(final_prompt)
print('-' * 20 + '结合上位问题和原问题回答' + '-' * 20)
print(f"final_prompt:\n {final_prompt}")
print('答案：\n' + result)

In [ ]:
# 5.假设性文档嵌入
import os

from langchain_chroma import Chroma
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

"""
核心思想
HyDE（Hypothetical Document Embeddings 假设性文档嵌入） 是一种无需真实文档的检索增强生成（RAG）技术。
其核心是通过模型先生成假设性答案，再根据该假设答案的嵌入向量去检索真实文档。

关键步骤
    1. 生成假设答案：
        让模型基于问题生成一个假设的答案（无需准确，只需语义相关）。
        示例：
        问题 → "如何训练一只猫用马桶？"
        假设答案 → "训练猫用马桶需要逐步引导，首先将猫砂盆靠近马桶..."
    2. 嵌入假设答案：
        将假设答案转换为向量（如用 OpenAI embeddings）。
    3. 向量检索：
        用该向量在数据库中检索真实相关的文档。
    4. 生成最终答案：
        结合检索到的真实文档生成可靠回答。
"""

# 1.文件路径
RESOURCE_DIR = "/mnt/c/大模型/智泊大模型全栈教程总结/02-教材整理 L2/代码/Langchain/6.langchain高级RAG/data/resources"
TXT_DOCUMENT_PATH = os.path.join(RESOURCE_DIR, "deepseek百度百科.txt")

# 2.模型初始化
llm = ChatTongyi(model="qwen-max")
embeddings_model = DashScopeEmbeddings(model="text-embedding-v1")

# 3.加载文档
loader = TextLoader(TXT_DOCUMENT_PATH, encoding='utf-8')
docs = loader.load()

# 4.文档向量化并存储
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=chunks,
                                    embedding=embeddings_model,
                                    collection_name="multi-query")
retriever = vectorstore.as_retriever()

# 4.生成假设答案
hyde_prompt = """
    请根据问题生成一个假设性答案（无需准确）：
    问题: {question}
    假设答案:
    """
question = "详细介绍DeepSeek"
hypothetical_answer = llm.invoke(hyde_prompt.format(question=question))
print(f"假设答案：\n {hypothetical_answer.content}")

# 5. 基于答案检索文档
retrieved_docs = vectorstore.similarity_search(hypothetical_answer.content, k=3)
print(f"召回文档：\n {retrieved_docs}")

# 6. 基于召回文档生成答案
final_prompt = f"""
    基于以下真实文档回答问题：
    文档: {retrieved_docs}
    问题: {question}
    答案:
    """
print(f"最终提示：\n{final_prompt}")
result = llm.invoke(final_prompt)
print('-' * 20 + '基于召回文档生成答案' + '-' * 20)
print(result)

In [ ]:
# 6.混合召回
import os

from langchain_chroma import Chroma
from langchain_classic.retrievers import EnsembleRetriever
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import RecursiveCharacterTextSplitter


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i + 1}:\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


# 1.文件路径
RESOURCE_DIR = "/mnt/c/大模型/智泊大模型全栈教程总结/02-教材整理 L2/代码/Langchain/6.langchain高级RAG/data/resources"
TXT_DOCUMENT_PATH = os.path.join(RESOURCE_DIR, "deepseek百度百科.txt")

# 2.模型准备
embeddings_model = DashScopeEmbeddings(model="text-embedding-v1")

# 3.数据加载
loader = TextLoader(TXT_DOCUMENT_PATH, encoding='utf-8')
docs = loader.load()

# 4.文档向量化并存储
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=chunks, embedding=embeddings_model, collection_name="mix"
)

# 5.文档召回
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
query = "相关评价"
vector_retriever_doc = vector_retriever.invoke(query)
print("\n\n向量召回文档：" + "==" * 100)
pretty_print_docs(vector_retriever_doc)

# 6.BM25检索
BM25_retriever = BM25Retriever.from_documents(chunks, k=3)
BM25Retriever_doc = BM25_retriever.invoke(query)
print("\n\n关键词召回文档：" + "==" * 100)
pretty_print_docs(BM25Retriever_doc)

# 7.混合检索
# 向量检索和关键词检索的权重各0.5，两者赋予相同的权重
retriever = EnsembleRetriever(retrievers=[BM25_retriever, vector_retriever], weights=[0.5, 0.5])
print("\n\n混合召回文档：" + "==" * 100)
pretty_print_docs(retriever.invoke(query))